Закоменченные строки пока не удалять (желательно существующие ячейки не изменять, а создавать новые)

In [42]:
# ячейка с импортом необходимого (исключить импорт в других ячейках)
from pyspark.sql import *
from pyspark import SparkContext, SparkConf
import json
import numpy as np
import requests
from pyspark.sql import dataframe
from pyspark.sql.types import *
import pandas as pd
from IPython.display import display, HTML

In [43]:
# ячека с запуском сессис spark (возможно переделать в функцию)
spark = SparkSession.builder \
    .master("local") \
    .appName("HistData") \
    .config("spark.debug.maxToStringFields", "1000") \
    .getOrCreate()


In [44]:
def getResourceDict (json):
    # по листу метрики из json данных возвращает словарь процессов
    list_data = json['data']['result'] # список словарей по процессам
    # предобработка - convert to float
#     for i in list_data:
#         for j in range(len(i["values"])):
#             i["values"][j][1] = float(i["values"][j][1])
    # values_list = list(map(lambda x: x["values"], list_data))
    resourceTypeDict = dict(map(lambda x: (x["metric"]["resource_type"], x["values"]), list_data))
    # iterObj = iter(resourceTypeDict)
    # print(list(iterObj))
    return resourceTypeDict

In [45]:
# Блок склейки df спарка путем создания новых df в самом спарке (string data)

# импорт временного json по API прометеуса
# response = requests.get('http://localhost:9090/api/v1/query?query=' + 'PIDMemory' + '[24h]')
# listData = response.json()

with open('query.json', 'r') as j:
    json_data = json.load(j)
    # print(json_data)

resourceTypeDict = getResourceDict(json_data)
column_list = list(resourceTypeDict.keys())
print("Count (processes x metric); ", len(column_list)) # 
data = list(resourceTypeDict.values())
col = ['Time'] + list(resourceTypeDict.keys())

df = None
for el in column_list[:5]: # изменять срез для получения большего кол-ва столбцов
    # df_schema = StructType([StructField("Time", FloatType(), True), StructField(el, DoubleType(), True)]) - почему-то отторгает такую схему
    if df is None:
        df = spark.createDataFrame(resourceTypeDict[el]).toDF("Time", el)
    else:
        df_right = spark.createDataFrame(resourceTypeDict[el]).toDF("Time", el)
        df = df.join(df_right, "Time", "right")

print(f"#### Columns: {len(df.columns)} #### {df.count()}")
df.printSchema()

Count (processes x metric);  3324
#### Columns: 6 #### 37
root
 |-- Time: double (nullable = true)
 |-- AGMService.exe_5208_1604522690.0_0: string (nullable = true)
 |-- AGMService.exe_5208_1604522690.0_1: string (nullable = true)
 |-- AGMService.exe_5208_1604522690.0_10: string (nullable = true)
 |-- AGMService.exe_5208_1604522690.0_11: string (nullable = true)
 |-- AGMService.exe_5208_1604522690.0_2: string (nullable = true)



In [46]:
def printDf(sprkDF): 
    # преобразует спарквский df в пандасовский для нормального визуального отображения
    newdf = sprkDF.toPandas()
    return HTML(newdf.to_html())

Ячейка ниже - копия логики создания объединенного датафрейма но на пандасе (1001 столбец генерит за несколько секунд) c дальнейшим переводом его в df спарка.
Для вывода на экран df спарка использовать функцию printDf 


In [66]:
# response = requests.get('http://localhost:9090/api/v1/query?query=' + 'PIDMemory' + '[24h]')
# listData = response.json()

with open('query.json', 'r') as j:
    json_data = json.load(j)
    # print(json_data)

resourceTypeDict = getResourceDict(json_data)
column_list = list(resourceTypeDict.keys())
print(len(column_list))
data = list(resourceTypeDict.values())
col = ['Time'] + list(resourceTypeDict.keys())

pdf = None
for el in column_list[:10]: # изменять срез для получения большего кол-ва столбцов
    if pdf is None:
        pdf = pd.DataFrame(resourceTypeDict[el], columns=["Time", el])
    else:
        pdf_right = pd.DataFrame(resourceTypeDict[el], columns=["Time", el])
        pdf = pdf.join(pdf_right.set_index("Time"), on ="Time")

pdf = pdf.astype(float)
spark.conf.set("spark.sql.debug.maxToStringFields", "1000")
df_spark = spark.createDataFrame(pdf)
print(f"#### Columns: {len(pdf.columns)} #### ")
df_spark.show()
# pdf
#printDf(df_spark)

3324
#### Columns: 11 #### 
+----------------+----------------------------------+----------------------------------+-----------------------------------+-----------------------------------+----------------------------------+----------------------------------+----------------------------------+----------------------------------+----------------------------------+----------------------------------+
|            Time|AGMService.exe_5208_1604522690.0_0|AGMService.exe_5208_1604522690.0_1|AGMService.exe_5208_1604522690.0_10|AGMService.exe_5208_1604522690.0_11|AGMService.exe_5208_1604522690.0_2|AGMService.exe_5208_1604522690.0_3|AGMService.exe_5208_1604522690.0_4|AGMService.exe_5208_1604522690.0_5|AGMService.exe_5208_1604522690.0_6|AGMService.exe_5208_1604522690.0_7|
+----------------+----------------------------------+----------------------------------+-----------------------------------+-----------------------------------+----------------------------------+----------------------------------+

In [75]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Loads data.
df_spark.select([c for c in df_spark.columns if c in ['AGMService.exe_5208_1604522690.0_0']]).show()
dataset

# Trains a k-means model.
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(dataset)

# Make predictions
predictions = model.transform(dataset)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

AnalysisException: cannot resolve '`AGMService.exe_5208_1604522690.0_0`' given input columns: [AGMService.exe_5208_1604522690.0_0, AGMService.exe_5208_1604522690.0_1, AGMService.exe_5208_1604522690.0_10, AGMService.exe_5208_1604522690.0_11, AGMService.exe_5208_1604522690.0_2, AGMService.exe_5208_1604522690.0_3, AGMService.exe_5208_1604522690.0_4, AGMService.exe_5208_1604522690.0_5, AGMService.exe_5208_1604522690.0_6, AGMService.exe_5208_1604522690.0_7, Time];;
'Project ['AGMService.exe_5208_1604522690.0_0]
+- LogicalRDD [Time#554, AGMService.exe_5208_1604522690.0_0#555, AGMService.exe_5208_1604522690.0_1#556, AGMService.exe_5208_1604522690.0_10#557, AGMService.exe_5208_1604522690.0_11#558, AGMService.exe_5208_1604522690.0_2#559, AGMService.exe_5208_1604522690.0_3#560, AGMService.exe_5208_1604522690.0_4#561, AGMService.exe_5208_1604522690.0_5#562, AGMService.exe_5208_1604522690.0_6#563, AGMService.exe_5208_1604522690.0_7#564], false
